# Creating new tables

In [8]:
import sqlite3
import pandas as pd

con = sqlite3.connect("university_professors.db")
cur = con.cursor()
cur.execute("""CREATE TABLE professors (
                firstname text,
                lastname text
               );"""
               )

con.close()

In [9]:
# connect to database
con = sqlite3.connect("university_professors.db")
cur = con.cursor()

In [10]:
pd.read_sql("""SELECT * 
               FROM professors
               ;""", 
            con)

,firstname,lastname


In [11]:
cur.execute("""CREATE TABLE universities (
                university_shortname text,
                university text,
                university_city text
               );"""
               )

In [12]:
pd.read_sql("""SELECT * 
               FROM universities
               ;""", 
            con)

,university_shortname,university,university_city


## Adding columns to existing tables is easy, especially if they're still empty.

To add columns you can use the following SQL query:

```
ALTER TABLE table_name
ADD COLUMN column_name data_type;
```

In [15]:
cur.execute("""ALTER TABLE professors
               ADD COLUMN university_shortname text
               ;"""
               )

In [16]:
pd.read_sql("""SELECT * 
               FROM professors
               ;""", 
            con)

,firstname,lastname,university_shortname


In [17]:
cur.execute("""CREATE TABLE affiliations (
                firstname text,
                lastname text,
                university_shortname text,
                function text,
                organisation text
               );"""
               )

- ### To rename columns:
```
ALTER TABLE table_name
RENAME COLUMN old_name TO new_name;
```

- ### To delete columns:
```
ALTER TABLE table_name
DROP COLUMN column_name;
```

In [18]:
cur.execute("""ALTER TABLE affiliations
               RENAME COLUMN organisation TO organization
               ;"""
               )

In [19]:
cur.execute("""ALTER TABLE affiliations
               DROP COLUMN university_shortname
               ;"""
               )

- ### For table deletion, you can use the simple command:

```
DROP TABLE table_name;
```